In [1]:
from datasets import load_dataset
imdb = load_dataset('imdb')

/Users/abhi/miniconda3/envs/hf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
imdb["train"][1]

{'text': '"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, 

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:20<00:00, 2447.20 examples/s]


In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
sentences = [
    "Hello, how are you?",
    "I am fine, thank you.",
    "Thank you very much!",
    "How's everything going?"
]

In [14]:
# tokenize the sentences
inputs = tokenizer(sentences, padding=True)

In [15]:
inputs

{'input_ids': [[101, 7592, 1010, 2129, 2024, 2017, 1029, 102, 0], [101, 1045, 2572, 2986, 1010, 4067, 2017, 1012, 102], [101, 4067, 2017, 2200, 2172, 999, 102, 0, 0], [101, 2129, 1005, 1055, 2673, 2183, 1029, 102, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0]]}

In [16]:
import torch

In [18]:
inputs['input_ids']

[[101, 7592, 1010, 2129, 2024, 2017, 1029, 102, 0],
 [101, 1045, 2572, 2986, 1010, 4067, 2017, 1012, 102],
 [101, 4067, 2017, 2200, 2172, 999, 102, 0, 0],
 [101, 2129, 1005, 1055, 2673, 2183, 1029, 102, 0]]

In [19]:
input_ids = torch.tensor(inputs['input_ids'])

In [23]:
input_ids

tensor([[ 101, 7592, 1010, 2129, 2024, 2017, 1029,  102,    0],
        [ 101, 1045, 2572, 2986, 1010, 4067, 2017, 1012,  102],
        [ 101, 4067, 2017, 2200, 2172,  999,  102,    0,    0],
        [ 101, 2129, 1005, 1055, 2673, 2183, 1029,  102,    0]])

In [24]:
input_ids.shape

torch.Size([4, 9])

In [25]:
attention_mask = torch.tensor(inputs['attention_mask'])